<a href="https://colab.research.google.com/github/plaban1981/Langchain_usecases/blob/main/Essay_Grader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install -qU openai langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 3.9 MB/s eta 0:00:00


In [2]:
import os
import openai
from random import choice
import json
import langchain
from langchain.prompts.base import StringPromptTemplate
from langchain.prompts import PromptTemplate,StringPromptTemplate
from langchain.agents import Tool, AgentExecutor, AgentOutputParser,LLMSingleActionAgent,initialize_agent
from langchain import OpenAI,LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.schema import AgentAction,AgentFinish
import re
from typing import List,Union
#
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
#
from langchain.prompts import PromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain.chat_models import ChatOpenAI
from time import time, sleep


from getpass import getpass
os.environ["OPENAI_API_KEY"] = getpass()



··········


In [33]:

def preprocess_function(request_data, content_type=None):
    with open(request_data,"r",encoding='utf-8') as f:
        context = f.read()
    print(context)
    #
    p_word_count_schema = ResponseSchema(name="Prescribed Word Count",
                                description = "This is Word count that has been prescribed as 300 Words ")
    #
    s_word_count_schema = ResponseSchema(name="Student Word Count",
                                    description = "This is the word count thats specifies the number of words present in the essay content specified within 3 backticks")
    #
    s_word_limit_schema = ResponseSchema(name="Try to stay within + or - 10 words of the prescribed word limit",
                                        description="This is the word limit that the student has adhered the word limit or exceeded the word limit")
    #
    title_schema = ResponseSchema(name="Title",
                                description="Title of the Essay in case provided")
    #
    content_schema = ResponseSchema(name="Content and Structure",
                                    description="This provides points between -5 to 10 based on the GRADING CRITERIA for CONTENT")
    #
    language_schema = ResponseSchema(name="Language",
                                    description="This provides the points based on the GRADING CRITERIA for LANGAUAGE AND GRAMMAR.Also provide spelling errors in RED and provide the correct answer next to the incorrect words in GREEN BOLD.")
    #
    context_schema = ResponseSchema(name="Corrected Context",
                                    description="This provides the grammatically corrected words for  the context provided in capital Letters .")
    #
    response_schemas = [p_word_count_schema,
                        s_word_count_schema,
                        s_word_limit_schema,
                        title_schema,
                        content_schema,
                        language_schema,
                        context_schema]
    #
    output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
    format_instructions = output_parser.get_format_instructions()
    print("\n")
    print(output_parser)
    print("\n")
    print(format_instructions)
    #
    template ="""
Evaluate the context of the essay specified in 3 bacticks on the following criteria:
ESSAY CRITERIA:
1. TITLE : The essay must have a title.
2. WORD LIMIT: The word count in the essay must be between 290 to 310 words
3. CONTENT : The content must have the matter as per the current developments. Government Schemes pertaining to a topic, allocation as per the budget, developments/achievements till now, way forward, etc
4. LANGAUAGE AND GRAMMAR: Errors of subject verb agreement, preposition, Spellings, appropriate spacing, capitalization and punctuations must be checked.

GRADING CRITERIA:
Please grade the essay on the below parameters and provide a Overall Grades as A+ or A or B+ or B or C or D.
1. LANGAUAGE AND GRAMMAR: Please check subject verb agreement errors.Also Spellings, appropriate spacing, capitalization and punctuations must be checked.
2. WORD LIMIT: The total word count is 300 words. Please  deduct -0.5 marks each for every increase or decrease of 10 words.
3.CONTENT : Please analyse the matter as per the current developments. Government Schemes pertaining to a topic, allocation as per the budget, developments/achievements till now, way forward, etc, whether they are mentioned or not.If the content lacks any of the points mentioned deduct -0.5 marks each.
4. TITLE : Please deduct 3 marks if either title or serial number of the chosen essay topic is not  mentioned.


ESSAY CONTENT:
```
{input}

```

Please provide the feedback in the format specified below :
FEEDBACK:

1.Prescribed word count: round 300 words

2.Student word count:  specify the number of words present in the essay content specified within 3 backticks

3.Try to stay within + OR - 10 words of the prescribed word limit : If the student word count is within a + or - 10 words then mention "Well Done" otherwise mention "Word Limit not adhered"

4.Title: If tile is present then mention " Title provided" otherwise mention "No Title provided"

5. Content and Structure: Provide points between -5 to 10 based on the GRADING CRITERIA for CONTENT

6. Language: Provide the points based on the GRADING CRITERIA for LANGAUAGE AND GRAMMAR.Also provide spelling errors in RED and provide the correct answer next to the incorrect words in GREEN BOLD.

7. Corrected Content :Please  provide the grammatically corrected sentences in the context provided within 3 backticks and
   specify the words that have been added to make grammar corrections within 3 single quote ''' .
for example:
In 2015, '''THE''' government came out with insurance sector schemes such as PM Suraksha Bima Yojana. It provided '''A''' basic savings bank deposit account with
no minimum deposit requirements. The poor '''LACK''' even the basic services necessary for a healthy life.

{format_instructions}

Please provide the response in JSON format with the key elements as

Prescribed word count
Student word count
Try to stay within + OR - 10 words of the prescribed word limit
Title
Content and Structure
Language
Corrected Context
"""
    #
    prompt = ChatPromptTemplate(
            messages=[
                HumanMessagePromptTemplate.from_template(template)
            ],
            input_variables=["input"],
            partial_variables={"format_instructions": format_instructions},
            output_parser=output_parser # here we add the output parser to the Prompt template
        )
    #
    return (prompt,context)


#
#
def predict_function(data, model_path):
    prompt,context = data
    api_key = os.environ["OPENAI_API_KEY"]
    #
    llm = ChatOpenAI(openai_api_key=api_key,temperature=0.0)
    #
    chain = LLMChain(llm=llm,prompt=prompt)
    #
    response = chain.predict_and_parse(input=context,history=None)
    #
    return response

#
def model_load_function(model_path):
    return model_path


#
def postprocess_function(predictions, content_type=None):
    return json.dumps({"response": predictions})




In [34]:
%%writefile request_data.txt
(c) Financial Inclusion and development of the poor

India is one of the countries with highly inequitable distribution of wealth.
A report pointed out, that only 1% of the wealthy Indians hold
90% of the total income of our country. India, due to its population
explosion has one of the lowest per capita .
The poor even the basic services necessary for a healthy life.
We ranked 132 in the HDI index, which signifies, that a lot of
groundwork needs to be done to improve even the basic Human
Development factors.

Financial Inclusion refers to providing basic financial services such
as banking, insurance, pension etc. to the poorest and the
vulnerable strata of society. Most of the rural poor lack the
access to basic financial services. The Union Government came
out with PM Jan Dhan Yojana in 2014 to provide banking services
to the poor. It provided  basic savings bank deposit account with
no minimum deposit requirements. It was the first step in bringing
a host of financial services thereafter.

In 2015, government came out with insurance sector schemes such as PM Suraksha Bima Yojana and PM Jeevan Jyoti Beema Yojana. The former was a Personal accident
insurance having coverage upto Rs 2 lacs for death at a premium
of Rs 20/-and the latter being a life insurance scheme at a premium
of Rs 436/- for Sum Insured of Rs 2 lacs. This followed with schemes
in the pension sector viz. Atal Pension Yojana, PM SVanidhi, PM Shram
Yogi Maan dhan and recent amendments in NPS.

All the above government initiatives point to the efforts made for
the financial inclusion of the poor. These initiatives have not only helped
poor gain access to basic financial services, but also for targeted
delivery of DBT and subsidy services directly in the bank accounts
of the beneficiaries.


Overwriting request_data.txt


In [35]:
txt_path = "request_data.txt"
data = preprocess_function(txt_path)

(c) Financial Inclusion and development of the poor

India is one of the countries with highly inequitable distribution of wealth.
A report pointed out, that only 1% of the wealthy Indians hold
90% of the total income of our country. India, due to its population
explosion has one of the lowest per capita .
The poor even the basic services necessary for a healthy life.
We ranked 132 in the HDI index, which signifies, that a lot of
groundwork needs to be done to improve even the basic Human
Development factors.

Financial Inclusion refers to providing basic financial services such
as banking, insurance, pension etc. to the poorest and the
vulnerable strata of society. Most of the rural poor lack the
access to basic financial services. The Union Government came
out with PM Jan Dhan Yojana in 2014 to provide banking services
to the poor. It provided  basic savings bank deposit account with
no minimum deposit requirements. It was the first step in bringing
a host of financial services there

In [36]:
predictions = predict_function(data,model_path=None)

In [37]:
predictions

{'Prescribed Word Count': '300',
 'Student Word Count': '244',
 'Try to stay within + or - 10 words of the prescribed word limit': 'Word Limit not adhered',
 'Title': 'No Title provided',
 'Content and Structure': '-2',
 'Language': '-2',
 'Corrected Context': '(c) Financial Inclusion and Development of the Poor\n\nIndia is one of the countries with highly inequitable distribution of wealth. A report pointed out that only 1% of the wealthy Indians hold 90% of the total income of our country. Due to its population explosion, India has one of the lowest per capita incomes. The poor lack even the basic services necessary for a healthy life. We are ranked 132 in the HDI index, which signifies that a lot of groundwork needs to be done to improve even the basic human development factors.\n\nFinancial Inclusion refers to providing basic financial services such as banking, insurance, pension, etc. to the poorest and most vulnerable strata of society. Most of the rural poor lack access to basic

In [32]:
postprocess_function(predictions, content_type=None)

'{"response": {"Prescribed Word Count": "300", "Student Word Count": "244", "Try to stay within + or - 10 words of the prescribed word limit": "Word Limit not adhered", "Title": "No Title provided", "Content and Structure": "-2", "Language": "-2", "Corrected Context": "(c) Financial Inclusion and Development of the Poor\\n\\nIndia is one of the countries with highly inequitable distribution of wealth. A report pointed out that only 1% of the wealthy Indians hold 90% of the total income of our country. Due to its population explosion, India has one of the lowest per capita incomes. The poor lack even the basic services necessary for a healthy life. We rank 132 in the HDI index, which signifies that a lot of groundwork needs to be done to improve even the basic human development factors.\\n\\nFinancial Inclusion refers to providing basic financial services such as banking, insurance, pension, etc. to the poorest and most vulnerable strata of society. Most of the rural poor lack access to

In [ ]:
## Test the script

"""
if __name__ == '__main__':
    txt_path = "test.txt"
    data = preprocess_function(txt_path)
    model_path = "./model_files"
    path = model_load_function(model_path)
    predictions = predict_function(data,path)
    out = postprocess_function(predictions)
    print(out)
"""

In [38]:
res = '(c) Financial Inclusion and Development of the Poor\n\nIndia is one of the countries with highly inequitable distribution of wealth. A report pointed out that only 1% of the wealthy Indians hold 90% of the total income of our country. Due to its population explosion, India has one of the lowest per capita incomes. The poor lack even the basic services necessary for a healthy life. We are ranked 132 in the HDI index, which signifies that a lot of groundwork needs to be done to improve even the basic human development factors.\n\nFinancial Inclusion refers to providing basic financial services such as banking, insurance, pension, etc. to the poorest and most vulnerable strata of society. Most of the rural poor lack access to basic financial services. The Union Government came out with the PM Jan Dhan Yojana in 2014 to provide banking services to the poor. It provided a basic savings bank deposit account with no minimum deposit requirements. It was the first step in bringing a host of financial services thereafter.\n\nIn 2015, the government came out with insurance sector schemes such as PM Suraksha Bima Yojana and PM Jeevan Jyoti Beema Yojana. The former was a personal accident insurance having coverage up to Rs 2 lakhs for death at a premium of Rs 20, and the latter being a life insurance scheme at a premium of Rs 436 for a sum insured of Rs 2 lakhs. This was followed by schemes in the pension sector, namely Atal Pension Yojana, PM Svandhi, PM Shram Yogi Maan Dhan, and recent amendments in NPS.\n\nAll the above government initiatives point to the efforts made for the financial inclusion of the poor. These initiatives have not only helped the poor gain access to basic financial services but also for targeted delivery of DBT and subsidy services directly into the bank accounts of the beneficiaries.'

In [40]:
original = """
(c) Financial Inclusion and development of the poor

India is one of the countries with highly inequitable distribution of wealth.
A report pointed out, that only 1% of the wealthy Indians hold
90% of the total income of our country. India, due to its population
explosion has one of the lowest per capita .
The poor even the basic services necessary for a healthy life.
We ranked 132 in the HDI index, which signifies, that a lot of
groundwork needs to be done to improve even the basic Human
Development factors.

Financial Inclusion refers to providing basic financial services such
as banking, insurance, pension etc. to the poorest and the
vulnerable strata of society. Most of the rural poor lack the
access to basic financial services. The Union Government came
out with PM Jan Dhan Yojana in 2014 to provide banking services
to the poor. It provided  basic savings bank deposit account with
no minimum deposit requirements. It was the first step in bringing
a host of financial services thereafter.

In 2015, government came out with insurance sector schemes such as PM Suraksha Bima Yojana and PM Jeevan Jyoti Beema Yojana. The former was a Personal accident
insurance having coverage upto Rs 2 lacs for death at a premium
of Rs 20/-and the latter being a life insurance scheme at a premium
of Rs 436/- for Sum Insured of Rs 2 lacs. This followed with schemes
in the pension sector viz. Atal Pension Yojana, PM SVanidhi, PM Shram
Yogi Maan dhan and recent amendments in NPS.

All the above government initiatives point to the efforts made for
the financial inclusion of the poor. These initiatives have not only helped
poor gain access to basic financial services, but also for targeted
delivery of DBT and subsidy services directly in the bank accounts
of the beneficiaries.
"""

In [90]:
prompt = """
Please comapre the contents of the text enclosed within 3 backticks ``` with the contents of the text enclosed within 3 curly braces {{{
and provide a comaprison between them. Provide comparison based on new words added.
CORRECTED text
```
<<response>>
```

ANSWER PROVIDE
{{{

<<original>>

}}}

RESPONSE:
"""

In [91]:
prompt = prompt.replace("<<response>>",res).replace("<<original>>",original)

In [55]:
openai.api_key =getpass()



··········
[{'role': 'system', 'content': 'Your are a helpful text comparison assistant.'}, {'role': 'user', 'content': '\nPlease comapre the contents of the text enclosed within 3 backticks ``` with the contents of the text enclosed within 3 curly braces {{{ \nand provide only only the words that is present in TEXT1 but not in TEXT2\nTEXT1\n```\n(c) Financial Inclusion and Development of the Poor\n\nIndia is one of the countries with highly inequitable distribution of wealth. A report pointed out that only 1% of the wealthy Indians hold 90% of the total income of our country. Due to its population explosion, India has one of the lowest per capita incomes. The poor lack even the basic services necessary for a healthy life. We are ranked 132 in the HDI index, which signifies that a lot of groundwork needs to be done to improve even the basic human development factors.\n\nFinancial Inclusion refers to providing basic financial services such as banking, insurance, pension, etc. to the poo

In [92]:
#
messages = [{"role":"system",
             "content":"Your are a helpful text comparison assistant.",
             },
            ]
#
messages.append({"role":"user","content":prompt})
print(messages)

[{'role': 'system', 'content': 'Your are a helpful text comparison assistant.'}, {'role': 'user', 'content': '\nPlease comapre the contents of the text enclosed within 3 backticks ``` with the contents of the text enclosed within 3 curly braces {{{ \nand provide a comaprison between them. Provide comparison based on new words added.\nCORRECTED text\n```\n(c) Financial Inclusion and Development of the Poor\n\nIndia is one of the countries with highly inequitable distribution of wealth. A report pointed out that only 1% of the wealthy Indians hold 90% of the total income of our country. Due to its population explosion, India has one of the lowest per capita incomes. The poor lack even the basic services necessary for a healthy life. We are ranked 132 in the HDI index, which signifies that a lot of groundwork needs to be done to improve even the basic human development factors.\n\nFinancial Inclusion refers to providing basic financial services such as banking, insurance, pension, etc. to

In [93]:
chat = openai.ChatCompletion.create(model="gpt-3.5-turbo",
                                        messages = messages,
                                        temperature=0,
                                        )

In [94]:
reply = chat.choices[0].message.content

In [70]:
print(reply)

The comparison between the two texts reveals the following new words added in TEXT2:

1. "India, due to its population explosion has one of the lowest per capita" - In TEXT1, the word "per capita" is followed by "incomes", while in TEXT2, it is followed by a period.

2. "The poor even the basic services necessary for a healthy life." - In TEXT1, the word "lack" is missing after "poor" in TEXT2.

3. "We ranked 132 in the HDI index, which signifies, that a lot of groundwork needs to be done to improve even the basic Human Development factors." - In TEXT1, the word "signifies" is missing after "which" in TEXT2.

4. "such as banking, insurance, pension etc." - In TEXT1, the word "etc." is missing after "pension" in TEXT2.

5. "The Union Government came out with PM Jan Dhan Yojana in 2014 to provide banking services to the poor." - In TEXT1, the word "out" is missing after "came" in TEXT2.

6. "It provided  basic savings bank deposit account with no minimum deposit requirements." - In TEXT1

In [76]:
print(reply)

The following words are missing in the corrected text:

- "income"
- "explosion"
- "even"
- "the"
- "basic"
- "services"
- "necessary"
- "ranked"
- "signifies"
- "Human"
- "Development"
- "such"
- "as"
- "etc."
- "the"
- "and"
- "the"
- "access"
- "to"
- "the"
- "the"
- "came"
- "out"
- "with"
- "to"
- "the"
- "It"
- "was"
- "the"
- "first"
- "step"
- "in"
- "bringing"
- "a"
- "host"
- "of"
- "thereafter"
- "2015"
- "government"
- "out"
- "with"
- "such"
- "as"
- "the"
- "former"
- "was"
- "a"
- "Personal"
- "accident"
- "upto"
- "lacs"
- "for"
- "at"
- "a"
- "premium"
- "of"
- "and"
- "the"
- "being"
- "a"
- "life"
- "at"
- "a"
- "premium"
- "of"
- "for"
- "Sum"
- "Insured"
- "of"
- "followed"
- "with"
- "schemes"
- "in"
- "the"
- "sector"
- "viz."
- "PM"
- "SVanidhi"
- "PM"
- "Shram"
- "Yogi"
- "Maan"
- "dhan"
- "and"
- "recent"
- "amendments"
- "in"
- "All"
- "the"
- "above"
- "government"
- "initiatives"
- "point"
- "to"
- "the"
- "efforts"
- "made"
- "for"
- "the"
- "of"
- "the"
-

In [95]:
print(reply)

Based on a comparison of the two texts, the following new words have been added in the corrected text:

1. "India, due to its population explosion has one of the lowest per capita."
2. "The poor even the basic services necessary for a healthy life."
3. "We ranked 132 in the HDI index, which signifies, that a lot of groundwork needs to be done to improve even the basic Human Development factors."
4. "The Union Government came out with PM Jan Dhan Yojana in 2014 to provide banking services to the poor."
5. "It provided basic savings bank deposit account with no minimum deposit requirements."
6. "It was the first step in bringing a host of financial services thereafter."
7. "In 2015, government came out with insurance sector schemes such as PM Suraksha Bima Yojana and PM Jeevan Jyoti Beema Yojana."
8. "The former was a Personal accident insurance having coverage upto Rs 2 lacs for death at a premium of Rs 20/-and the latter being a life insurance scheme at a premium of Rs 436/- for Sum In